# Setting

- We have k chunks to estimate
- Each chunk i already has $m_i$ past results $R_{ij} \sim Lap(1/n_i \epsilon_{ij})$
- Linear combination: $Q_{ij} := \frac{n_i}{n} \gamma_{ij}(q_i + R_{ij})$
- We need $\sum_j \gamma_{ij} = 1$ for each chunk $i$ to have an unbiased estimate
- Goal: output an estimate $Q = Q_{10} + \dots + Q_{1m_1} +\dots + Q_{k0} + \dots + Q_{km_k}$
- Accuracy constraint: $\Pr[|Q-q|>\alpha] < \beta$
- Simplifying budget constraint: $R_{i0} \sim Lap(1/n_\epsilon)$ (same budget for everyone!). That's a weird constraint if we have chunks that are already good, let's see if we can set some epsilons to infinity. In the general case we could try an ILP or even some gradient descent on the budget vector?

In [1]:
import numpy as np

from precycle.utils.utility_theorems import get_epsilon_isotropic_laplace_concentration, binary_search

In [2]:
def minimum_variance(epsilons, noises, chunk_sizes):
    n_chunks = len(epsilons)
    chunk_noises = np.zeros(n_chunks)
    for chunk_id in range(n_chunks):
        # The variance has an extra 2/n_i^2 factor but it doesn't matter at the chunk level
        laplace_coefficients = epsilons[chunk_id] ** 2
        # See optimal variance reduction and lemma in Overleaf
        laplace_coefficients = laplace_coefficients / sum(laplace_coefficients)
        chunk_noises[chunk_id] = np.dot(laplace_coefficients, noises[chunk_id])
        
    chunk_coefficients = chunk_sizes / sum(chunk_sizes)
    aggregated_noise_total = np.dot(chunk_coefficients, chunk_noises)
    return aggregated_noise_total

In [3]:
# TODO: given epsilon and past results, find beta

def monte_carlo_beta(existing_epsilons, chunk_sizes, fresh_epsilon, alpha, N=1_000_000):
    
    # Add fresh epsilon
    epsilons = [
        np.append(eps_by_chunk, fresh_epsilon) for eps_by_chunk in existing_epsilons
    ]
    
    # TODO: heuristic to drop some chunks?
    
    # Vectorized code with a batch dimension corresponding to N
    n_chunks = len(epsilons)
    n = sum(chunk_sizes)
    chunk_noises = np.zeros((N, n_chunks))
    for chunk_id in range(n_chunks):
        # The final laplace scale (Q_ij), already scaled by n_i/n * eps^2/sum(eps^2)
        single_chunk_laplace_scale = epsilons[chunk_id] / (n * np.sum(epsilons[chunk_id] ** 2))
        laplace_scale = np.repeat([single_chunk_laplace_scale], N, axis=0)
        laplace_noises = np.random.laplace(scale=laplace_scale)
        
        
        # Optimal average for that chunk, N times
        chunk_noises[:, chunk_id] = np.sum(laplace_noises, axis=1)
        
    aggregated_noise_total = np.sum(chunk_noises, axis=1)
    beta = np.sum(aggregated_noise_total > alpha) / N
    return beta


In [4]:
existing_epsilons = [
    np.array([1, 0.5, 2]),
    np.array([0.3, 2.1])
]

chunk_sizes = [100, 200]
fresh_epsilon = 0.1
alpha = 0.001

monte_carlo_beta(existing_epsilons, chunk_sizes, fresh_epsilon, alpha, N=10_000_000)

0.3510764

In [5]:
# TODO: binary search for epsilon

def binary_search_monte_carlo(existing_epsilons, chunk_sizes, alpha, beta, N=1_000_000):
    get_beta_fn = lambda eps: monte_carlo_beta(existing_epsilons=existing_epsilons, chunk_sizes=chunk_sizes, fresh_epsilon=eps, alpha=alpha, N=N)
    
    # Worst case: ignore all existing epsilons, use loose concentration bound over the k chunks
    epsilon_high = get_epsilon_isotropic_laplace_concentration(a=alpha, b=beta, n=sum(chunk_sizes), k=len(chunk_sizes))
    
    return binary_search(get_beta_fn=get_beta_fn, beta=beta, epsilon_high=epsilon_high)

In [6]:
binary_search_monte_carlo(existing_epsilons, chunk_sizes, alpha=alpha, beta=0.01, N=100_000)

0 < 24.976507591763784 < 49.95301518352757 gives beta=0.001. Target 0.01
0 < 12.488253795881892 < 24.976507591763784 gives beta=0.03252. Target 0.01
12.488253795881892 < 18.73238069382284 < 24.976507591763784 gives beta=0.00687. Target 0.01
12.488253795881892 < 15.610317244852366 < 18.73238069382284 gives beta=0.01466. Target 0.01
15.610317244852366 < 17.171348969337604 < 18.73238069382284 gives beta=0.01003. Target 0.01
17.171348969337604 < 17.951864831580224 < 18.73238069382284 gives beta=0.00837. Target 0.01
17.171348969337604 < 17.561606900458912 < 17.951864831580224 gives beta=0.00952. Target 0.01
17.171348969337604 < 17.366477934898256 < 17.561606900458912 gives beta=0.00947. Target 0.01
17.171348969337604 < 17.26891345211793 < 17.366477934898256 gives beta=0.00953. Target 0.01
17.171348969337604 < 17.220131210727764 < 17.26891345211793 gives beta=0.00952. Target 0.01
17.171348969337604 < 17.195740090032686 < 17.220131210727764 gives beta=0.00981. Target 0.01
17.171348969337604 <

17.17289025859497

In [7]:
binary_search_monte_carlo(existing_epsilons, chunk_sizes=[10,290], alpha=alpha, beta=0.01, N=1_000_000)

0 < 24.976507591763784 < 49.95301518352757 gives beta=0.001303. Target 0.01
0 < 12.488253795881892 < 24.976507591763784 gives beta=0.032188. Target 0.01
12.488253795881892 < 18.73238069382284 < 24.976507591763784 gives beta=0.006517. Target 0.01
12.488253795881892 < 15.610317244852366 < 18.73238069382284 gives beta=0.014758. Target 0.01
15.610317244852366 < 17.171348969337604 < 18.73238069382284 gives beta=0.009937. Target 0.01
15.610317244852366 < 16.390833107094984 < 17.171348969337604 gives beta=0.01206. Target 0.01
16.390833107094984 < 16.781091038216296 < 17.171348969337604 gives beta=0.010906. Target 0.01
16.781091038216296 < 16.97622000377695 < 17.171348969337604 gives beta=0.010356. Target 0.01
16.97622000377695 < 17.07378448655728 < 17.171348969337604 gives beta=0.010199. Target 0.01
17.07378448655728 < 17.122566727947444 < 17.171348969337604 gives beta=0.009768. Target 0.01
17.07378448655728 < 17.09817560725236 < 17.122566727947444 gives beta=0.010066. Target 0.01
17.09817560

17.101163364581666

In [8]:
# TODO: Why do we get slightly different epsilons? What are our exact guarantees?

## Aggregation with no VR

Let's start with chunks of same size and no past epsilons

In [9]:
import plotly.express as px
import pandas as pd

In [9]:

total_size = 100_000

d = []

for beta in [1e-4, 1e-3]:
    for alpha in [0.001, 0.01]:
        for k in [1,2,3,4,5,6,7,10,12,14,16,18,20]:
            existing_epsilons = [np.array([])]*k
            chunk_sizes = [100_000 // k] * k
            for bound in ["monte_carlo", "concentration"]:
                if bound == "monte_carlo":
                    try:
                        epsilon = binary_search_monte_carlo(existing_epsilons, chunk_sizes, alpha=alpha, beta=beta, N=1_000_000)
                    except AssertionError as e:
                        print(e)
                        epsilon = np.NAN
                else:
                    epsilon = get_epsilon_isotropic_laplace_concentration(a=alpha, b=beta, n=total_size, k=k)
                d.append(dict(
                    alpha=alpha,
                    beta=beta,
                    k=k,
                    epsilon=epsilon,
                    bound=bound        
                ))
df = pd.DataFrame(d)

KeyboardInterrupt: 

In [ ]:
df

,alpha,beta,k,epsilon,bound
0,0.001,0.0001,1,0.086257,monte_carlo
1,0.001,0.0001,1,0.092103,concentration
2,0.001,0.0001,2,0.103128,monte_carlo
3,0.001,0.0001,2,0.280113,concentration
4,0.001,0.0001,3,0.118174,monte_carlo
...,...,...,...,...,...
99,0.010,0.0010,16,0.031192,concentration
100,0.010,0.0010,18,0.019340,monte_carlo
101,0.010,0.0010,18,0.033084,concentration
102,0.010,0.0010,20,0.020297,monte_carlo


In [ ]:
px.line(df, x='k', y='epsilon', color='bound', facet_col="alpha", facet_row="beta", title=f'epsilon vs k')

## VR on a single chunk

In [14]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
#TODO: check if epsilon=0 can work first

In [17]:
from precycle.utils.utility_theorems import get_epsilon_vr_monte_carlo

In [33]:

existing_epsilons = [np.array([5])]
chunk_sizes = [1_000]
alpha = 0.01
beta = 0.01
get_epsilon_vr_monte_carlo(existing_epsilons, chunk_sizes, alpha=alpha, beta=beta, N=1_000_000)

Sufficient fresh eps for chunk 0: 44.440972086577936
Fresh epsilons: [44.44097209]
Chunk 0 laplace scale: [2.5000000e-06 2.2220486e-05]
Fresh epsilons: [22.22048604]
Chunk 0 laplace scale: [9.63855422e-06 4.28346719e-05]
0 < 22.220486043288968 < 44.440972086577936 gives beta=0.0. Target 0.01
Fresh epsilons: [11.11024302]
Chunk 0 laplace scale: [3.36842105e-05 7.48479530e-05]
0 < 11.110243021644484 < 22.220486043288968 gives beta=0.0. Target 0.01
Fresh epsilons: [5.55512151]
Chunk 0 laplace scale: [8.95104895e-05 9.94483291e-05]
0 < 5.555121510822242 < 11.110243021644484 gives beta=0.0. Target 0.01
Fresh epsilons: [2.77756076]
Chunk 0 laplace scale: [1.52835821e-04 8.49021556e-05]
0 < 2.777560755411121 < 5.555121510822242 gives beta=0.0. Target 0.01
Fresh epsilons: [1.38878038]
Chunk 0 laplace scale: [1.85675431e-04 5.15724789e-05]
0 < 1.3887803777055605 < 2.777560755411121 gives beta=0.0. Target 0.01
Fresh epsilons: [0.69439019]
Chunk 0 laplace scale: [1.96215569e-04 2.72500332e-05]
0 

(8.083765730876246e-11, array([1.]))

In [43]:
d = []


for beta in [1e-4, 1e-3]:
    for alpha in [0.001, 0.01]:
        for existing_eps in [0.0001, 0.001, 0.01, 0.1, 0.5, 1, 10]:
            existing_epsilons = [np.array([existing_eps])]
            chunk_sizes = [10_000]
            for bound in ["monte_carlo"]:
                if bound == "monte_carlo":
                    try:
                        epsilon, _ = get_epsilon_vr_monte_carlo(existing_epsilons, chunk_sizes, alpha=alpha, beta=beta, N=1_000_000)                    
                    except AssertionError as e:
                        print(e)
                        epsilon = np.NAN
                else:
                    epsilon = get_epsilon_isotropic_laplace_concentration(a=alpha, b=beta, n=total_size, k=k)
                d.append(dict(
                    alpha=alpha,
                    beta=beta,
                    existing_eps=existing_eps,
                    epsilon=epsilon,
                    bound=bound        
                ))
df = pd.DataFrame(d)

Sufficient fresh eps for chunk 0: 4472.135954999578
Fresh epsilons: [4472.135955]
Chunk 0 laplace scale: [5.00000000e-16 2.23606798e-08]
Fresh epsilons: [2236.0679775]
Chunk 0 laplace scale: [2.00000000e-15 4.47213595e-08]
0 < 2236.067977499789 < 4472.135954999578 gives beta=0.0. Target 0.0001
Fresh epsilons: [1118.03398875]
Chunk 0 laplace scale: [8.00000000e-15 8.94427191e-08]
0 < 1118.0339887498944 < 2236.067977499789 gives beta=0.0. Target 0.0001
Fresh epsilons: [559.01699437]
Chunk 0 laplace scale: [3.20000000e-14 1.78885438e-07]
0 < 559.0169943749472 < 1118.0339887498944 gives beta=0.0. Target 0.0001
Fresh epsilons: [279.50849719]
Chunk 0 laplace scale: [1.28000000e-13 3.57770876e-07]
0 < 279.5084971874736 < 559.0169943749472 gives beta=0.0. Target 0.0001
Fresh epsilons: [139.75424859]
Chunk 0 laplace scale: [5.12000000e-13 7.15541753e-07]
0 < 139.7542485937368 < 279.5084971874736 gives beta=0.0. Target 0.0001
Fresh epsilons: [69.8771243]
Chunk 0 laplace scale: [2.04800000e-12 1.

In [40]:
df

,alpha,beta,existing_eps,epsilon,bound
0,0.001,0.0001,0.001,8.657871e-01,monte_carlo
1,0.001,0.0001,0.100,8.359324e-01,monte_carlo
2,0.001,0.0001,0.500,5.373851e-01,monte_carlo
3,0.001,0.0001,1.000,6.355287e-11,monte_carlo
4,0.001,0.0001,10.000,6.355272e-11,monte_carlo
5,0.010,0.0001,0.001,8.631675e-02,monte_carlo
6,0.010,0.0001,0.100,8.038873e-11,monte_carlo
7,0.010,0.0001,0.500,8.038873e-11,monte_carlo
8,0.010,0.0001,1.000,8.038871e-11,monte_carlo
9,0.010,0.0001,10.000,8.038672e-11,monte_carlo


In [45]:
px.line(df, log_x=True, x='existing_eps', y='epsilon', color='bound', facet_col="alpha", facet_row="beta", title=f'fresh epsilon depending on existing eps (with Variance Reduction)')

## Now, let's do VR on multiple chunks


In [50]:
existing_epsilons = [np.array([5, 10]), np.array([0.01])]
chunk_sizes = [1_000, 2_000]
alpha = 0.01
beta = 0.01
get_epsilon_vr_monte_carlo(existing_epsilons, chunk_sizes, alpha=alpha, beta=beta, N=1_000_000)

Sufficient fresh eps for chunk 0: 23.273733406281565
Sufficient fresh eps for chunk 1: 25.819887038224365
Fresh epsilons: [25.81988704 25.81988704]
Chunk 0 laplace scale: [2.10526342e-06 4.21052685e-06 1.08715328e-05]
Chunk 1 laplace scale: [5.00000000e-09 1.29099435e-05]
Fresh epsilons: [12.90994352 12.90994352]
Chunk 0 laplace scale: [5.71428620e-06 1.14285724e-05 1.47542224e-05]
Chunk 1 laplace scale: [1.99999910e-08 2.58198754e-05]
0 < 12.909943519112183 < 25.819887038224365 gives beta=0.0. Target 0.01
Fresh epsilons: [6.45497176 6.45497176]
Chunk 0 laplace scale: [1.00000004e-05 2.00000008e-05 1.29099440e-05]
Chunk 1 laplace scale: [7.99998200e-08 5.16396579e-05]
0 < 6.454971759556091 < 12.909943519112183 gives beta=0.0. Target 0.01
Fresh epsilons: [3.22748588 3.22748588]
Chunk 0 laplace scale: [1.23076924e-05 2.46153849e-05 7.94458072e-06]
Chunk 1 laplace scale: [3.19996976e-07 1.03278572e-04]
0 < 3.2274858797780457 < 6.454971759556091 gives beta=0.0. Target 0.01
Fresh epsilons: 

(0.13001988982596113, array([1., 1.]))

In [51]:
existing_epsilons = [np.array([5, 10]), np.array([0.1])]
chunk_sizes = [1_000, 2_000]
alpha = 0.01
beta = 0.01
get_epsilon_vr_monte_carlo(existing_epsilons, chunk_sizes, alpha=alpha, beta=beta, N=1_000_000)

Sufficient fresh eps for chunk 0: 23.273733406281565
Sufficient fresh eps for chunk 1: 25.81969532482261
Fresh epsilons: [25.81969532 25.81969532]
Chunk 0 laplace scale: [2.10528975e-06 4.21057950e-06 1.08715880e-05]
Chunk 1 laplace scale: [5.00000000e-08 1.29098477e-05]
Fresh epsilons: [12.90984766 12.90984766]
Chunk 0 laplace scale: [5.71433469e-06 1.14286694e-05 1.47542381e-05]
Chunk 1 laplace scale: [1.99991000e-07 2.58185335e-05]
0 < 12.909847662411305 < 25.81969532482261 gives beta=0.0. Target 0.01
Fresh epsilons: [6.45492383 6.45492383]
Chunk 0 laplace scale: [1.00000375e-05 2.00000750e-05 1.29098961e-05]
Chunk 1 laplace scale: [7.99820040e-07 5.16277744e-05]
0 < 6.454923831205653 < 12.909847662411305 gives beta=0.0. Target 0.01
Fresh epsilons: [3.22746192 3.22746192]
Chunk 0 laplace scale: [1.23077065e-05 2.46154130e-05 7.94453081e-06]
Chunk 1 laplace scale: [3.19697885e-06 1.03181275e-04]
0 < 3.2274619156028264 < 6.454923831205653 gives beta=0.0. Target 0.01
Fresh epsilons: [1

(0.07168155068110973, array([1., 1.]))

In [52]:
existing_epsilons = [np.array([5, 10]), np.array([0.2])]
chunk_sizes = [1_000, 2_000]
alpha = 0.01
beta = 0.01
get_epsilon_vr_monte_carlo(existing_epsilons, chunk_sizes, alpha=alpha, beta=beta, N=1_000_000)

Sufficient fresh eps for chunk 0: 23.273733406281565
Sufficient fresh eps for chunk 1: 25.81911436642757
Fresh epsilons: [25.81911437 25.81911437]
Chunk 0 laplace scale: [2.10536953e-06 4.21073907e-06 1.08717554e-05]
Chunk 1 laplace scale: [1.00000000e-07 1.29095572e-05]
Fresh epsilons: [12.90955718 12.90955718]
Chunk 0 laplace scale: [5.71448164e-06 1.14289633e-05 1.47542855e-05]
Chunk 1 laplace scale: [3.99928013e-07 2.58144678e-05]
0 < 12.909557183213785 < 25.81911436642757 gives beta=0.0. Target 0.01
Fresh epsilons: [6.45477859 6.45477859]
Chunk 0 laplace scale: [1.00001500e-05 2.00003000e-05 1.29097508e-05]
Chunk 1 laplace scale: [1.59856129e-06 5.15917961e-05]
0 < 6.4547785916068925 < 12.909557183213785 gives beta=0.0. Target 0.01
Fresh epsilons: [3.2273893 3.2273893]
Chunk 0 laplace scale: [1.23077491e-05 2.46154982e-05 7.94437955e-06]
Chunk 1 laplace scale: [6.37589910e-06 1.02887543e-04]
0 < 3.2273892958034462 < 6.4547785916068925 gives beta=0.0. Target 0.01
Fresh epsilons: [1

(9.392939088294068e-11, array([1., 1.]))

In [53]:
existing_epsilons = [np.array([5, 10]), np.array([0.1, 0.1])]
chunk_sizes = [1_000, 2_000]
alpha = 0.01
beta = 0.01
get_epsilon_vr_monte_carlo(existing_epsilons, chunk_sizes, alpha=alpha, beta=beta, N=1_000_000)

Sufficient fresh eps for chunk 0: 23.273733406281565
Sufficient fresh eps for chunk 1: 25.819501673476708
Fresh epsilons: [25.81950167 25.81950167]
Chunk 0 laplace scale: [2.10531634e-06 4.21063269e-06 1.08716438e-05]
Chunk 1 laplace scale: [5.00000000e-08 5.00000000e-08 1.29097508e-05]
Fresh epsilons: [12.90975084 12.90975084]
Chunk 0 laplace scale: [5.71438368e-06 1.14287674e-05 1.47542539e-05]
Chunk 1 laplace scale: [1.99982002e-07 1.99982002e-07 2.58171781e-05]
0 < 12.909750836738354 < 25.819501673476708 gives beta=0.0. Target 0.01
Fresh epsilons: [6.45487542 6.45487542]
Chunk 0 laplace scale: [1.00000750e-05 2.00001500e-05 1.29098477e-05]
Chunk 1 laplace scale: [7.99640162e-07 7.99640162e-07 5.16157762e-05]
0 < 6.454875418369177 < 12.909750836738354 gives beta=0.0. Target 0.01
Fresh epsilons: [3.22743771 3.22743771]
Chunk 0 laplace scale: [1.23077207e-05 2.46154414e-05 7.94448039e-06]
Chunk 1 laplace scale: [3.19396341e-06 3.19396341e-06 1.03083179e-04]
0 < 3.2274377091845885 < 6.

(9.393079989777728e-11, array([1., 1.]))

I don't know how to plot it but it looks good I think!